<a href="https://colab.research.google.com/github/dasalgadob/handson-machine-learning/blob/master/3Imageretrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
from tensorflow import stack
#from keras.datasets import mnist
from keras.datasets import mnist
import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector
import os

Using TensorFlow backend.


### Chapter 1 Previous code to chapter 3

In [1]:
## Loading
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [0]:
input_size = 784
no_classes = 10
batch_size = 100
total_batches = 400

In [0]:
## Placeholders for input data and targets
x_input = tf.placeholder(tf.float32, shape=[None, input_size])
y_input = tf.placeholder(tf.float32, shape=[None, no_classes])

In [0]:
weights = tf.Variable(tf.random_normal([input_size, no_classes]))
bias = tf.Variable(tf.random_normal([no_classes]))

In [0]:
logits = tf.matmul(x_input, weights) + bias

In [0]:
softmax_cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(
labels=y_input, logits=logits)
loss_operation = tf.reduce_mean(softmax_cross_entropy)
optimiser = tf.train.GradientDescentOptimizer(
learning_rate=0.5).minimize(loss_operation)

In [0]:
### Training the model with data
session = tf.Session()
session.run(tf.global_variables_initializer())

for batch_no in range(total_batches):
  mnist_batch = mnist_data.train.next_batch(batch_size)
  _, loss_value = session.run([optimiser, loss_operation], feed_dict={
  x_input: mnist_batch[0],
  y_input: mnist_batch[1]
  })
  #print(loss_value)

In [19]:
predictions = tf.argmax(logits, 1)
correct_predictions = tf.equal(predictions, tf.argmax(y_input, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_predictions,
tf.float32))
test_images, test_labels = mnist_data.test.images, mnist_data.test.labels
accuracy_value = session.run(accuracy_operation, feed_dict={
x_input: test_images,
y_input: test_labels
})
print('Accuracy : ', accuracy_value)
session.close()

Accuracy :  0.8425


In [0]:
### Multilayer convolutional network

def add_variable_summary(tf_variable, summary_name):
  with tf.name_scope(summary_name + '_summary'):
    mean = tf.reduce_mean(tf_variable)
    tf.summary.scalar('Mean', mean)
    with tf.name_scope('standard_deviation'):
      standard_deviation = tf.sqrt(tf.reduce_mean( tf.square(tf_variable - mean)))
    tf.summary.scalar('StandardDeviation', standard_deviation)
    tf.summary.scalar('Maximum', tf.reduce_max(tf_variable))
    tf.summary.scalar('Minimum', tf.reduce_min(tf_variable))
    tf.summary.histogram('Histogram', tf_variable)

In [0]:
x_input_reshape = tf.reshape(x_input, [-1, 28, 28, 1],
name='input_reshape') ## -1 refers that the batch size could be any

## Embedding visualization

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
print(mnist_data[::1])

(<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f8f0f858f28>, <tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f8f0f88b240>, <tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f8f0f88b390>)


In [0]:
no_embedding_data = 1000
embedding_variable = tf.Variable(stack( mnist_data.test.images[:no_embedding_data], axis=0), trainable= False)

### MNIST test data to metadata file for visualization

In [0]:
metadata_path = '/metadata.tsv'
work_dir = ''

with open(metadata_path, 'w') as metadata_file:
  for i in range(no_embedding_data):
    metadata_file.write('{}\n'.format( np.nonzero(mnist_data.test.labels[::1])[1:][0][i]))
    
    
projector_config =projector.ProjectorConfig()
embedding_projection = projector_config.embeddings.add()
embedding_projection.tensor_name = embedding_variable.name
embedding_projection.metadata_path = metadata_path
embedding_projection.sprite.image_path = os.path.join( work_dir + 'mnist_10k_sprite.png')
embedding_projection.sprite.single_image_dim.extend([28,28])

In [0]:
#projector.visualize_embeddings(train_summary_writer, projector_config)
#tf.train.Saver().save(session, '/tmp/train/model.ckpt', global_step=1)